In [72]:
import pandas as pd
import pyodbc


In [73]:
# Read the data from access database

dbPath = "./go_sales.accdb"
sales_db = pyodbc.connect(r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + dbPath)

In [74]:
# Read the tables from access database
order_details = pd.read_sql("SELECT * FROM order_details", sales_db)
product = pd.read_sql("SELECT * FROM product", sales_db)
order_header = pd.read_sql("SELECT * FROM order_header", sales_db)
order_method = pd.read_sql("SELECT * FROM order_method", sales_db)


c:\Users\manny\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\manny\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\manny\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\manny\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\io\

In [75]:
# Merge the tables
order_details = pd.merge(order_details, product, on="PRODUCT_NUMBER")
order_details = pd.merge(order_details, order_header, on="ORDER_NUMBER")
order_details = pd.merge(order_details, order_method, on="ORDER_METHOD_CODE")

In [76]:
# Create a new column
order_details["QUANTITY"] =  order_details["QUANTITY"].astype(float).where(order_details["QUANTITY"].notnull(), 0)
order_details["UNIT_PRICE"] = order_details["UNIT_PRICE"].astype(float).where(order_details["UNIT_PRICE"].notnull(), 0)
order_details["total_revenue"] = order_details["QUANTITY"] * order_details["UNIT_PRICE"]

In [77]:
order_details["ORDER_DATE"] = pd.to_datetime(order_details["ORDER_DATE"])
order_details["YEAR"] = order_details["ORDER_DATE"].dt.year
# Group by the order method type
order_details.groupby(["YEAR", "PRODUCT_NAME", "ORDER_METHOD_EN", "UNIT_PRICE"])["total_revenue"].sum()

YEAR    PRODUCT_NAME         ORDER_METHOD_EN  UNIT_PRICE
2020.0  .28                  E-mail           40.00          5080.00
                                              40.52          3079.52
                                              127.27        53198.86
                                              134.11        10594.69
                             Fax              0.00              0.00
                                                              ...   
2022.0  TrailChef Water Bag  Telephone        0.00              0.00
                                              2.00            184.00
                                              4.00            408.00
                                              6.00          14304.00
                                              204.00         1020.00
Name: total_revenue, Length: 4084, dtype: float64